In [ ]:
# 语言模型，提问范式与 Token
# 在本章中，我们将和您分享大型语言模型（LLM）的工作原理、训练方式以及分词器（tokenizer）等细节对 LLM 
# 输出的影响。我们还将介绍 LLM 的提问范式（chat format），这是一种指定系统消息（system message）和
# 用户消息（user message）的方式，让您了解如何利用这种能力。

In [ ]:
# !pip install openai
# !pip install langchain
# !pip install --upgrade tiktoken

In [1]:
from openai import OpenAI
def get_completion_gpt(prompt, model="gpt-4o-mini", temperature=0): 
    messages = [{"role": "user", "content": prompt}] # 消息队列
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 值越低则输出文本随机性越低
    )
    return response.choices[0].message.content

In [ ]:
# 尝试向模型提问并得到结果
# LLM 可以通过使用监督学习来构建，通过不断预测下一个词来学习。 并且，给定一个大的训练集，有数百亿甚至更多的词，你可
# 以创建一个大规模的训练集，你可以从一 句话或一段文本的一部分开始，反复要求语言模型学习预测下一个词是什么
# LLM 主要分为两种类型：基础语言模型（Base LLM）和越来越受欢迎的指令微调语言模型（Instruction Tuned LLM）。基础
# 语言模型通过反复预测下一个词来训练，因此如果我们给它一个 Prompt，比如“从前有一只独角兽”，它可能通过逐词预测来完成一
# 个关于独角兽在魔法森林中与其他独角兽朋友们生活的故事。
# 然而，这种方法的缺点是，如果您给它一个 Prompt，比如“中国的首都是哪里？”，很可能它数据中有一段互联网上关于中国的测验问
# 题列表。这时，它可能会用“中国最大的城市是什么？中国的人口是多少？”等等来回答这个问题。但实际上，您只是想知道中国的首都是
# 什么，而不是列举所有这些问题。然而，指令微调语言模型会尝试遵循 Prompt，并给出“中国的首都是北京”的回答。
# 那么，如何将基础语言模型转变为指令微调语言模型呢？这就是训练一个指令微调语言模型（例如ChatGPT）的过程。首先，您需要在大量数
# 据上训练基础语言模型，因此需要数千亿个单词，甚至更多。这个过程在大型超级计算系统上可能需要数月时间。训练完基础语言模型后，您会
# 通过在一小部分示例上进行进一步的训练，使模型的输出符合输入的指令。例如，您可以请承包商帮助您编写许多指令示例，并对这些指令的正
# 确回答进行训练。这样就创建了一个用于微调的训练集，让模型学会在遵循指令的情况下预测下一个词是什么。
# 之后，为了提高语言模型输出的质量，常见的方法是让人类对许多不同输出进行评级，例如是否有用、是否真实、是否无害等。然后，您可以进
# 一步调整语言模型，增加生成高评级输出的概率。这通常使用强化学习中的人类反馈（RLHF）技术来实现。相较于训练基础语言模型可能需要
# 数月的时间，从基础语言模型到指令微调语言模型的转变过程可能只需要数天时间，使用较小规模的数据集和计算资源。

In [ ]:
response = get_completion_gpt("What is the capital of China?")
print(response)

In [2]:
response = get_completion_gpt("中国的首都是哪里？")
print(response)

中国的首都是北京。


In [ ]:
# Tokens
# 到目前为止对 LLM 的描述中，我们将其描述为一次预测一个单词，但实际上还有一个更重要的技术细节。即 LLM 
# 实际上并不是重复预测下一个单词，而是重复预测下一个 token 。当 LLM 接收到输入时，它将将其转换为一系
# 列 token，其中每个 token 都代表常见的字符序列。例如，对于 "Learning new things is fun!" 这句话
# ，每个单词都被转换为一个 token ，而对于较少使用的单词，如 "Prompting as powerful developer tool"，
# 单词 "prompting" 会被拆分为三个 token，即"prom"、"pt"和"ing"。(子词单元)
# 当您要求 ChatGPT 颠倒 "lollipop" 的字母时，由于分词器（tokenizer） 将 "lollipop" 分解为三个 
# token，即 "l"、"oll"、"ipop"，因此 ChatGPT 难以正确输出字母的顺序。您可以通过在字母之间添加连字符或
# 空格的方式，使分词器将每个字母分解为单独的 token，从而帮助 ChatGPT 更好地认识单词中的每个字母并正确输出它们。

In [3]:
# 为了更好展示效果，这里就没有翻译成中文的 Prompt
# 注意这里的字母翻转出现了错误，吴恩达老师正是通过这个例子来解释 token 的计算方式
# GPT-4 在处理这个任务时已经改进了错误。吴恩达的例子反映了早期版本模型可能存在的局限性，而 GPT-4 
# 在 token 切分和任务理解上已经显著优化。所以这里反转正确
response = get_completion_gpt("Take the letters in lollipop \
and reverse them")
print(response)

Reversing the letters in "lollipop" gives you "popillol."


In [4]:
response = get_completion_gpt("""Take the letters in l-o-l-l-i-p-o-p and reverse them""")
print(response)

Reversing the letters in "l-o-l-l-i-p-o-p" gives you "p-o-p-i-l-l-o-l".


In [ ]:
# 对于英文输入，一个 token 一般对应 4 个字符或者四分之三个单词；对于中文输入，一个 token 一般对应一个或半个词。
# 不同模型有不同的 token 限制，需要注意的是，这里的 token 限制是输入的 Prompt 和输出的 completion 的 token 
# 数之和，因此输入的 Prompt 越长，能输出的 completion 的上限就越低。
# ChatGPT3.5-turbo 的 token 上限是 4096。

In [ ]:
# Helper function 辅助函数 (提问范式)
# 下面是课程中用到的辅助函数。 下图是 OpenAI 提供的一种提问范式，接下来吴恩达老师就是在演示如何利用这种范式进行更好的提问

In [5]:
# System 信息用于指定模型的规则，例如设定、回答准则等，而 assistant 信息就是让模型完成的具体指令
# 系统信息相当于给模型设定个框框,因为模型训练时应该是基于大量数据,这时设置到一定范围
# 而助理信息是模型基于多轮对话的上下文生成的回复
client = OpenAI()
def get_completion_from_messages(messages, 
                                 model="gpt-4o-mini", 
                                 temperature=0.5, 
                                 max_tokens=500):
    '''
    封装一个支持更多参数的自定义访问 gpt-4o-mini 的函数
    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-4o-mini(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0.5，增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message.content

In [ ]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [6]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理，并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

在蓝色的海洋中，游着一条小鲸，  
她快乐地嬉戏，像风中的纸鸥翎。  
波浪轻轻拍打，水花四处飞溅，  
小鲸鱼唱歌，声声如晨钟响叮当。

“来吧，朋友们，跟我一起舞动！  
在阳光的照耀下，让心情永远熏染！  
跳跃的海豚，欢快而灵动，  
我们在海底，编织梦想的画卷！”

她游过珊瑚，嬉戏在泡沫中，  
螃蟹们欢呼，海星们来捧场。  
小鲸鱼的笑声在海里荡漾，  
快乐的旋律，永不消亡！


In [ ]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

In [7]:
# 长度控制
messages =  [  
{'role':'system',
 'content':'你的所有答复只能是一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

从前有一只快乐的小鲸鱼，它每天在大海里唱歌、跳舞，与朋友们一起探险，享受海洋的美丽，直到有一天它遇到了一个迷路的小海龟，决定帮助它找到回家的路。


In [ ]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)


In [8]:
# 以上结合
messages =  [  
{'role':'system',
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答，只回答一句话'},    
{'role':'user',
 'content':'写一个关于快乐的小鲸鱼的故事'},
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

在波光粼粼的蓝色海洋里，有一只快乐的小鲸鱼，唱着欢快的歌，和朋友们一起旋转、跳跃，享受着无忧无虑的海洋乐趣！


In [ ]:
client = OpenAI()

In [9]:
def get_completion_and_token_count(messages, 
                                   model="gpt-4o-mini", 
                                   temperature=0, 
                                   max_tokens=500):
    """
    使用 OpenAI 的 gpt-4o-mini 模型生成聊天回复，并返回生成的回复内容以及使用的 token 数量。
    参数:
    messages: 聊天消息列表。
    model: 使用的模型名称。默认为"gpt-4o-mini"。
    temperature: 控制生成回复的随机性。值越大，生成的回复越随机。默认为 0。
    max_tokens: 生成回复的最大 token 数量。默认为 500。
    返回:
    content: 生成的回复内容。
    token_dict: 包含'prompt_tokens'、'completion_tokens'和'total_tokens'的字典，分别表示提示的 
    token 数量、生成的回复的 token 数量和总的 token 数量。
    """
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    content = response.choices[0].message.content
    token_dict = {
    'prompt_tokens':response.usage.prompt_tokens,
    'completion_tokens':response.usage.completion_tokens,
    'total_tokens':response.usage.total_tokens
    }
    return content, token_dict

In [10]:
response

'在波光粼粼的蓝色海洋里，有一只快乐的小鲸鱼，唱着欢快的歌，和朋友们一起旋转、跳跃，享受着无忧无虑的海洋乐趣！'

In [ ]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages,temperature=0.7)
print(response)

In [12]:
token_dict

{'prompt_tokens': 46, 'completion_tokens': 167, 'total_tokens': 213}

In [11]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

在蓝色的海洋，波浪轻轻摇，  
有一只小鲸鱼，快乐又骄傲。  
它在水中跳跃，像星星在舞，  
喷出水花，闪烁着光芒如珠。

“嗨，朋友们！”它欢快地叫，  
“来和我一起，畅游这海潮！  
我们在珊瑚间，嬉戏又玩耍，  
在阳光下游弋，快乐无比呀！”

小鲸鱼的笑声，传遍每个角落，  
在海洋的怀抱，幸福永不落。  
无论风浪如何，它总是微笑，  
因为在这大海，它是最快乐的宝！


In [13]:
print(token_dict)

{'prompt_tokens': 46, 'completion_tokens': 167, 'total_tokens': 213}


In [14]:
messages =  [  
{'role':'system', 
 'content':'你是一个助理， 并以 Seuss 苏斯博士的风格作出回答。只回答一句话。'},    
{'role':'user', 
 'content':'就快乐的小鲸鱼为主题给我写一首短诗'},  
] 
response, token_dict = get_completion_and_token_count(messages)
print(response)

在蓝色的海洋里，快乐的小鲸鱼，  
跳跃翻滚，欢声笑语，真是太美妙！  
它唱着歌，水花四溅，  
每个波浪都在为它欢呼，真是无比快乐的时光！


In [15]:
token_dict

{'prompt_tokens': 51, 'completion_tokens': 64, 'total_tokens': 115}

In [ ]:
# 最后，我们认为 Prompt 对 AI 应用开发的革命性影响仍未得到充分重视。在传统的监督机器学习工作流中，如果想要构建一
# 个可以将餐厅评论分类为正面或负面的分类器，首先需要获取一大批带有标签的数据，可能需要几百个，这个过程可能需要几周，
# 甚至一个月的时间。接着，您需要在这些数据上训练一个模型，找到一个合适的开源模型，并进行模型的调整和评估，这个阶段
# 可能需要几天、几周，甚至几个月的时间。最后，您可能需要使用云服务来部署模型，将模型上传到云端，并让它运行起来，才能
# 最终调用您的模型。整个过程通常需要一个团队数月时间才能完成。
# 相比之下，使用基于 Prompt 的机器学习方法，当您有一个文本应用时，只需提供一个简单的 Prompt 就可以了。这个过程可能
# 只需要几分钟，如果需要多次迭代来得到有效的 Prompt 的话，最多几个小时即可完成。在几天内（尽管实际情况通常是几个小时）
# ，您就可以通过 API 调用来运行模型，并开始使用。一旦您达到了这个步骤，只需几分钟或几个小时，就可以开始调用模型进行推
# 理。因此，以前可能需要花费六个月甚至一年时间才能构建的应用，现在只需要几分钟或几个小时，最多是几天的时间，就可以使用 
# Prompt 构建起来。这种方法正在极大地改变 AI 应用的快速构建方式。
# 需要注意的是，这种方法适用于许多非结构化数据应用，特别是文本应用，以及越来越多的视觉应用，尽管目前的视觉技术仍在发展中。
# 但它并不适用于结构化数据应用，也就是那些处理 Excel 电子表格中大量数值的机器学习应用。然而，对于适用于这种方法的应用，
# AI 组件可以被快速构建，并且正在改变整个系统的构建工作流。构建整个系统可能仍然需要几天、几周或更长时间，但至少这部分可以
# 更快地完成。
# 下一个章中，我们将展示如何利用这些组件来评估客户服务助手的输入。 这将是本课程中构建在线零售商客户服务助手的更完整示例的一部分。